# Cartographie de la culture de betterave au Bénin

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pprint import pp
from datetime import datetime
from pathlib import Path
import yaml
from tqdm import tqdm

from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pcse.util import WOFOST71SiteDataProvider
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse.fileinput import YAMLAgroManagementReader
from pcse.models import Wofost72_WLP_FD

from nevergrad.functions.irrigation.common_path import IRRIGATION_DATA_DIR
from nevergrad.functions.irrigation.irrigation import get_soil_data

In [ ]:
 yaml_agro_template = """
- 2006-01-01:
    CropCalendar:
        crop_name: sugarbeet
        variety_name: Sugarbeet_603
        crop_start_date: 2006-03-31
        crop_start_type: emergence
        crop_end_date: 2006-10-20
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null
"""


### Mise en place du simulateur

In [ ]:
def aggregate_crop_leaf_growth(latitude: float, longitude: float) -> float:
    # paramètre de culture
    cropfile = "https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/"
    cropdata = YAMLCropDataProvider(repository=cropfile)

    #paramètre de sol
    soildata = get_soil_data()

    #paramètre de site
    sitedata = WOFOST71SiteDataProvider(WAV=100)

    #combinaison des trois premiers params
    parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

    #paramètre de météo
    wdp = NASAPowerWeatherDataProvider(latitude=latitude, longitude=longitude)

    #paramètre de plante et d'agrogestion
    agromanagement = yaml.safe_load(yaml_agro_template)

    #Initialisation du modèle
    crop_evolution_model = Wofost72_WLP_FD(parameters, wdp, agromanagement)
    
    # Execution du modèle
    crop_evolution_model.run_till_terminate()
    output = crop_evolution_model.get_output()
    df_output = pd.DataFrame(output)
    
    return df_output["LAI"].sum()


### Construction de la liste des villes et leurs coordonnées géographiques
Télécharger le fichier ici: https://data.humdata.org/dataset/geoboundaries-admin-boundaries-for-benin?force_layout=desktop

In [ ]:
benin = gpd.read_file(
    Path.cwd() / "geoBoundaries-BEN-ADM2-all" / "geoBoundaries-BEN-ADM2_simplified.shp",
)

### Phase d'éxecution du modèle sur les différentes localités

In [ ]:
aggregated_crop_growth = []

for town in tqdm(benin["geometry"].centroid):
    longitude = town.x
    latitude = town.y
    print(f"{longitude=}", f"{latitude=}")
    growth_metric = aggregate_crop_leaf_growth(latitude=latitude, longitude=longitude)
    aggregated_crop_growth.append(
        {"Growth_metric": growth_metric}
    )

In [ ]:
df_crop_growth_per_city = pd.DataFrame(aggregated_crop_growth)
df_crop_growth_per_city

In [ ]:
df = pd.concat([benin, df_crop_growth_per_city], axis=1)
df

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 10))
benin.plot(color='white', edgecolor='black', ax=ax)

df.plot(ax=ax, column="Growth_metric", legend=True)

plt.show()
fig.savefig("Sugarbeet_603.jpeg", dpi=100)